In [136]:
import numpy as np

In [137]:
import katarchive

import datetime

import logging
logging.root.setLevel(logging.ERROR)


In [138]:
def findh5(det):   # uses detail list to get h5 name
    pos=det.find("Name:")
    if pos >0  :
        pr2=det[pos+37:pos+53].split()
        h5name=pr2[0].lstrip("/ng")
    return h5name

In [139]:
def findproj(text):
    found='Unknown'        # default
    chx=text.find('prop')
    if chx >0  :
        pr2=text[chx+11:chx+38].split()
        code=pr2[0]
        found=code.replace("=","")
    else:
        if text.find('maser') >0:
            found="Maser monitoring"
            #print text
    return found

In [140]:
def listpr(x):
    """
    takes a list and prints as string
    """
    li=" ".join(map(str,x))
    return li

In [141]:
today=datetime.datetime.today()
ago=datetime.timedelta(-14)   # 14 days is a fortnight
lastweek=today+ago
print today,lastweek
begin=lastweek.strftime(format="%d/%m/%Y")
end=today.strftime(format="%d/%m/%Y")

2016-03-31 13:07:26.298066 2016-03-17 13:07:26.298066


In [142]:
f = katarchive.search_archive(startdate=begin)
start = datetime.datetime.strptime(f[-1].__str__().split('\'')[3],'%Y-%m-%d %H:%M:%S %Z')
end = datetime.datetime.strptime(f[0].__str__().split('\'')[3],'%Y-%m-%d %H:%M:%S %Z')

Searched using: CAS.ProductTypeName:(KatFile) AND CAS.ProductTransferStatus:RECEIVED AND StartTime:[2016-03-16T22:00:00Z TO 2016-03-31T21:59:59Z]
Returning 101 results


In [143]:
#print 'First file in run: ' , start.isoformat()
#print 'Last file in run: ' , end.isoformat()

In [144]:
for fi in f:
    txts=fi.metadata.InstructionSet
    p=findproj(txts)
    print fi.metadata.ExperimentID, p, fi.metadata.Duration, fi.metadata.Description, len(fi.metadata.Antennas)


2 20131211NO-01 32460.45 PKS1510-089 5
2 COMM_Atten 56.0 Auto atten 5
2 COMM_RFI_SP 9073.1 RFI Scan in c16n25M4k 5
2 COMM_Atten 40.27 Auto atten 5
2 COMM_Atten 105.01 Auto atten 5
2 20130822NO-01 14740.2 W2332_5056 5
2 COMM_Atten 106.01 Auto atten 5
2 COMM_Atten 105.01 Auto atten 5
2 COMM_Point_LM 18108.44 Pointing Observation 5
2 COMM_Atten 106.01 Auto atten 5
2 COMM_Bandpass_TF 21670.3 Bandpass Calibrators 5
2 COMM_Atten 110.01 Auto atten 5
2 20140729RT-01 21575.3 N3_Observe_2_1822Mhz 5
2 COMM_Atten 66.01 Auto atten 5
2 20150825LR-01 32140.44 reticulum2 5
2 COMM_RFI_SP 7393.59 Basic RFI Scan: RFI SCAN 5
2 COMM_Atten 95.01 Auto atten 5
2 20140729RT-01 21785.3 S2_Observe_2 6
2 COMM_Atten 92.01 Auto atten 6
2 COMM_Image_NO 7470.1 Imaging Calibration 6
2 COMM_Atten 109.01 Auto atten 6
2 COMM_Bandpass_TF 7380.1 Bandpass Calibrators 6
2 COMM_Atten 66.01 Auto atten 6
2 20150825LR-01 32650.45 reticulum2 6
2 COMM_Atten 40.0 Auto atten 6
2 20130822NO-01 14620.2 W2332_5056 6
2 COMM_Atten 112.01

Only select observations that are longer than 1 hrs then extract the project id from the script parameters. If there is no project id then allocated No_project_id 

In [145]:
sbd = [] #Experiment ID
pid = [] #project ID
fnd = [] #Filename
dud = [] #Duration
antsd = [] #number of antennas
scrd = [] #script  -NOT USED NOW

for fi in f:
    if fi.metadata.Duration < 3600: #only pick hour long or greater observations for analysis
        continue
    # now try extract the project-id from the instruction-set string
    """
    script = fi.metadata.InstructionSet
    scrd.append(script)
    spl = script.split()
    """
    txts=fi.metadata.InstructionSet
    p=findproj(txts)
    if p == "Unknown":
        p="Maser Monitoring"
    sbd.append(fi.metadata.ExperimentID)
    pid.append(p)
    fnd.append(fi.metadata.Filename)
    dud.append(fi.metadata.Duration)
    antsd.append(len(fi.metadata.Antennas))


In [146]:
pi = np.array(pid)
sb = np.array(sbd)
du = np.array(dud)
ants = np.array(antsd)

p_cent = (sum(du)/((end.toordinal()-start.toordinal())*24*3600))*100

print 'Percentage system usage: ', np.floor(p_cent),'%'


Percentage system usage:  53.0 %


In [147]:
a_id = []
a_time = []
for p in set(pi):
    ind = pi == p
    print p,round(sum(du[ind])/3600,2)
    a_id.append(p)
    a_time.append(round(sum(du[ind])/3600,2))

20141111IM-01 4.14
COMM_Delay_TF 3.18
COMM_Tipping_NO 1.07
20150217RT-01 13.05
20130822NO-01 8.16
COMM_Bandpass_TF 19.78
20141002MB-01 10.81
20140729RT-01 41.92
20150825LR-01 24.25
20141219SS-01 5.9
COMM_Image_NO 6.23
COMM_RFI_SP 7.87
COMM_Point_LM 5.03
20131211NO-01 9.02
20150821LR-01 3.08
20141125NO-01 2.13


In [148]:
ind = np.argsort(a_id)
for i in ind:
    print ("%-16s %5.1f %d") %(a_id[i],a_time[i],ants[i])

20130822NO-01      8.2 5
20131211NO-01      9.0 6
20140729RT-01     41.9 5
20141002MB-01     10.8 5
20141111IM-01      4.1 5
20141125NO-01      2.1 6
20141219SS-01      5.9 6
20150217RT-01     13.1 5
20150821LR-01      3.1 6
20150825LR-01     24.2 6
COMM_Bandpass_TF  19.8 5
COMM_Delay_TF      3.2 5
COMM_Image_NO      6.2 6
COMM_Point_LM      5.0 6
COMM_RFI_SP        7.9 6
COMM_Tipping_NO    1.1 5


Find the unique project ids and determine the total observation duration.

Sort ID and time for display

In [149]:
for fi in f:
    x= repr(fi)
    pr=x.split(",")
    print pr[1][10:12],findh5(fi.metadata.Details) , fi.metadata.Description, listpr(fi.metadata.Antennas)

30 1459371606.h5 PKS1510-089 ant1 ant2 ant3 ant4 ant5
30 1459371442.h5 Auto atten ant1 ant2 ant3 ant4 ant5
30 1459360568.h5 RFI Scan in c16n25M4k ant1 ant2 ant3 ant4 ant5
30 1459360374.h5 Auto atten ant1 ant2 ant3 ant4 ant5
30 1459359889.h5 Auto atten ant1 ant2 ant3 ant4 ant5
30 1459326145.h5 W2332_5056 ant1 ant2 ant3 ant4 ant5
30 1459325892.h5 Auto atten ant1 ant2 ant3 ant4 ant5
29 1459265092.h5 Auto atten ant1 ant2 ant3 ant4 ant5
29 1459246394.h5 Pointing Observation ant1 ant2 ant3 ant4 ant5
29 1459246087.h5 Auto atten ant1 ant2 ant3 ant4 ant5
29 1459217810.h5 Bandpass Calibrators ant1 ant2 ant3 ant4 ant5
29 1459217669.h5 Auto atten ant1 ant2 ant3 ant4 ant5
28 1459196058.h5 N3_Observe_2_1822Mhz ant1 ant2 ant3 ant4 ant5
28 1459195960.h5 Auto atten ant1 ant2 ant3 ant4 ant5
28 1459163772.h5 reticulum2 ant1 ant2 ant3 ant4 ant5
28 1459156314.h5 Basic RFI Scan: RFI SCAN ant1 ant2 ant3 ant4 ant5
28 1459155735.h5 Auto atten ant1 ant2 ant3 ant4 ant5
28 1459125195.h5 S2_Observe_2 ant1 ant2 ant

In [150]:
f.sort(key=lambda k: k.metadata.StartTime)

In [156]:
for fi in f:
    x= repr(fi)
    pr=x.split(",")
    nrant=len(fi.metadata.Antennas) # number of antennas
    proj=findproj(fi.metadata.InstructionSet)
    if fi.metadata.Description.find('Auto') <0 and  fi.metadata.Description.find('RFI') <0 and \
      fi.metadata.Description.find('Pointing') <0  and  fi.metadata.Description.find('Delay') <0  and \
      fi.metadata.Description.find('Tipping') <0  and nrant>4 :  # not auto attenuate, more than 5 antenna
        print("%s/%s  %-18s %s") %(fi.metadata.FileLocation,findh5(fi.metadata.Details),proj,fi.metadata.Description )

/var/kat/archive/data/comm/2016/03/17/1458191116.h5  20141002MB-01      SS433
/var/kat/archive/data/comm/2016/03/17/1458202149.h5  20150217RT-01      NGC_1097 - 1850MHz
/var/kat/archive/data/comm/2016/03/18/1458316130.h5  COMM_Bandpass_TF   Bandpass Calibrators
/var/kat/archive/data/comm/2016/03/18/1458328565.h5  20140729RT-01      N3_Observe_1_1822Mhz
/var/kat/archive/data/comm/2016/03/20/1458475752.h5  20150217RT-01      NGC_1097 - 1600MHz
/var/kat/archive/data/comm/2016/03/21/1458561944.h5  20150821LR-01      GRB151021A
/var/kat/archive/data/comm/2016/03/21/1458573476.h5  20141219SS-01      W2332 Low
/var/kat/archive/data/comm/2016/03/21/1458588610.h5  20150217RT-01      NGC_1097 - 1350MHz
/var/kat/archive/data/comm/2016/03/22/1458623136.h5  COMM_Image_NO      Imaging Calibration
/var/kat/archive/data/comm/2016/03/24/1458788411.h5  20141002MB-01      SS433
/var/kat/archive/data/comm/2016/03/24/1458845520.h5  20140729RT-01      N3_Observe_2
/var/kat/archive/data/comm/2016/03/25/14588